In [91]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import datetime
import pandas as pd
import tweepy
import math
import glob
from tweepy import TweepError
import cPickle as pickle

In [92]:
# edit these three variables
user = 'potus'

# only edit these if you're having problems
delay = 1  # time to wait on each page load before reading the page

In [93]:
try:
    #raise IOError("Forcing")
    with open(user+'.pickle', 'rb') as handle:
        tweetCutoff = pickle.load(handle)
except IOError:
    if user == 'vp':
        tweetCutoff = 927930606222274560
    elif user == 'realdonaldtrump':
        tweetCutoff = 927673257230327808
    elif user == 'potus':
        tweetCutoff = 928056523766497280
    else:
        raise ValueError("Not implemented yet")

In [94]:
tweet_selector = 'li.js-stream-item'
user = user.lower()
ids = []
id_selector = '.time a.tweet-timestamp'


In [95]:
keys = {'consumer_key' : 'znPXxNt7LcofyAYpuMOdfO23Q',
        'consumer_secret' : 'MGrKG8lv6r20Sa47gbheDWs3jhJXWVMkMp3VJxxRP74LQd5U6D',
        'access_token' : '935858594431553538-UidHBRgEZC2Q7hhEcbbE1HaSKrzy8JR',
        'access_token_secret' : 'P3mNnM67hWzFBg9uAJJRkv7PWSscx2NsPkJUJU04HaHsa'
}

In [96]:
driver = webdriver.Chrome()  # options are Chrome() Firefox() Safari()
driver.get('https://twitter.com/'+user)
quit = False
n = 0
ids = []
while not quit:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)
    found_tweets = driver.find_elements_by_css_selector(tweet_selector)
    for tweet in found_tweets[n:]:
        if tweet.find_element_by_css_selector('div.tweet').get_attribute('data-retweet-id'):
            ids_ = tweet.find_element_by_css_selector('div.tweet').get_attribute('data-retweet-id')
            ids.append(ids_)
        else:
            ids_ = tweet.find_element_by_css_selector('div.tweet').get_attribute('data-tweet-id')
            ids.append(ids_)
        if int(ids_) == int(tweetCutoff):
            quit = True
            break
            
    n = len(found_tweets)
    
    
    
if found_tweets[0].find_element_by_css_selector('div.tweet').get_attribute('data-retweet-id'):
    tweetCutoff = found_tweets[0].find_element_by_css_selector('div.tweet').get_attribute('data-retweet-id')
else:
    tweetCutoff = found_tweets[0].find_element_by_css_selector('div.tweet').get_attribute('data-tweet-id')    

driver.quit()

In [97]:
consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

print('total ids: {}'.format(len(ids)))

all_data = []
start = 0
end = 100
limit = len(ids)
i = math.ceil(limit / 100.)

total ids: 4


In [98]:
def isRetweet(entry):
    return 'retweeted_status' in entry.keys()

In [99]:
for go in xrange(int(i)):
    print('currently getting {} - {}'.format(start, end))
    if go:
        sleep(10)  # needed to prevent hitting API rate limit
    id_batch = ids[start:end]
    start += 100
    end += 100
    tweets = api.statuses_lookup(id_batch)
    for tweet in tweets:
        all_data.append(dict(tweet._json))


currently getting 0 - 100


In [100]:
parsed_data = []
for i, d in enumerate(all_data):
    text =  d['text'].encode('utf-8').strip()
    parsed_data.append((isRetweet(d), d['created_at'],text))

In [101]:
try:
    df_old = pd.read_csv(user+'.csv')
except IOError:
    df_old = pd.DataFrame()

In [102]:
df = pd.DataFrame(parsed_data, columns = ['is_retweet', 'created_at', 'text'])

In [103]:
final_df = pd.concat([df_old, df]).drop_duplicates()

In [104]:
final_df.to_csv(user+'.csv', index=False)

In [105]:
with open(user+'.pickle', 'wb') as handle:
    pickle.dump(tweetCutoff, handle, protocol=pickle.HIGHEST_PROTOCOL)